<a href="https://colab.research.google.com/github/alokranjan04/FileSaver.js/blob/master/testingWithSheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade -q gspread
!pip install gspread-dataframe
!pip install flask

In [0]:
import numpy as np
import pandas as pd
import gspread 
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [0]:
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
sheet1 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1O_RbV63LpZK7KSA6nPfisRJBGX5Rm8Cq_v3gfp3TC5o/edit#gid=0')
sheet2 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1BqMAqF5U4z5m-qRXD9BspZG5Ro_jh3x2a80KOOup_nU/edit#gid=0')
sheet3 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1dKh-1js_kMscAyUrML2TVAAZgBI4laugr-HoYr7xCBg/edit#gid=0')
sheet4 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1KLIRl47Q1bS3uhV0fgbqdctwjbz86R4ShxOQuc_iL7Q/edit#gid=0')
sheet5 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Rj2Q_bzw3wQ-LhfwULIwSDHl9cUVVr1m28zHx_ngKVo/edit#gid=1256653836')
sheet6 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1E7ZyPJzqU8Sw05e01AV6HckXh5uU-CgE8SAsxY08snU/edit#gid=1256653836')
sheet7 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1YW39Plivd870gR2PcrTDCQgce2n87kW8qo1Ir-tRV8M/edit#gid=1256653836')


ws1 = sheet1.worksheet('Sheet2')
ws2 = sheet2.worksheet('Sheet2')
ws3 = sheet3.worksheet('Sheet2')
ws4 = sheet4.worksheet('Sheet2')
ws5 = sheet5.worksheet('Sheet2')
ws6 = sheet6.worksheet('Sheet2')
ws7 = sheet7.worksheet('Sheet2')

data_df1 = get_as_dataframe(ws1)
data_df2 = get_as_dataframe(ws2)
data_df3 = get_as_dataframe(ws3)
data_df4 = get_as_dataframe(ws4)
data_df5 = get_as_dataframe(ws5)
data_df6 = get_as_dataframe(ws6)
data_df7 = get_as_dataframe(ws7)
df1 = data_df1[['Activity','Activity Time','LDAP']]
df2 = data_df2[['Activity','Activity Time','LDAP']]
df3 = data_df3[['Activity','Activity Time','LDAP']]
df4 = data_df4[['Activity','Activity Time','LDAP']]
df5 = data_df5[['Activity','Activity Time','LDAP']]
df7 = data_df6[['Activity','Activity Time','LDAP']]
df8 = data_df7[['Activity','Activity Time','LDAP']]
df6 = df1.append(df2).append(df3).append(df4).append(df5).append(df7).append(df8)
import time
import datetime
df6["ActivityTimeStamp"] =   df6["Activity Time"].apply( lambda x:    (time.mktime(datetime.datetime.strptime(x, "%Y/%m/%d %H:%M:%S").timetuple())  if ',' not in repr(x) else  (time.mktime(datetime.datetime.strptime(x, "%m/%d/%Y, %I:%M:%S %p").timetuple()) if (('AM' in repr(x)) | ('PM' in repr(x)) ) else  time.mktime(datetime.datetime.strptime(x, "%d/%m/%Y, %H:%M:%S").timetuple()))) if  isinstance(x, str) else 0)



In [0]:
df6 = df6.drop_duplicates()

In [0]:
def convertTime(counter):
    if (counter < 60 ):
      return str(counter)+" Sec"
    elif (counter < 3600 & counter >= 60 ):
      return str(int(counter/60))+" Min "+str(int(counter%60))+" Sec"
    elif (counter >= 3600 ):
      return str(int(counter/3600))+" Hour "+str(int((counter/3600 - int(counter/3600))*60))+" Min "+str(int(((counter/3600 - int(counter/3600))*60 - int((counter/3600 - int(counter/3600))*60))*60))+" Sec"

def getLdap(ldap, dfday):
    dfldap = dfday[dfday['LDAP'] == ldap]
    dfldap['TimeDiff'] =  dfldap['ActivityTimeStamp'].diff()
    startTime =  dfldap['Activity Time'].iloc[0] if (len(dfldap[dfldap['TimeDiff']> 3600*8]) ==0) else dfldap[dfldap['TimeDiff']> 3600*8]['Activity Time'].iloc[0]
    endTime = dfldap['Activity Time'].iloc[-1]
    stTime =  ((time.mktime(datetime.datetime.strptime(startTime, "%Y/%m/%d %H:%M:%S").timetuple())  if ',' not in repr(startTime) else  (time.mktime(datetime.datetime.strptime(startTime, "%m/%d/%Y, %I:%M:%S %p").timetuple()) if (('AM' in repr(startTime)) | ('PM' in repr(startTime)) ) else  time.mktime(datetime.datetime.strptime(startTime, "%d/%m/%Y, %H:%M:%S").timetuple()))) if  isinstance(startTime, str) else 0)
    enTime =  ((time.mktime(datetime.datetime.strptime(endTime, "%Y/%m/%d %H:%M:%S").timetuple())  if ',' not in repr(endTime) else  (time.mktime(datetime.datetime.strptime(endTime, "%m/%d/%Y, %I:%M:%S %p").timetuple()) if (('AM' in repr(endTime)) | ('PM' in repr(endTime)) ) else  time.mktime(datetime.datetime.strptime(endTime, "%d/%m/%Y, %H:%M:%S").timetuple()))) if  isinstance(endTime, str) else 0)
    return {'ldap': ldap, 'startTime': startTime, 'End Time': endTime, 'Total Time':(enTime - stTime) }
        

def getData(date1):
  time1 = time.mktime(datetime.datetime.strptime(date1+", 6:30:00 AM", "%m/%d/%Y, %I:%M:%S %p").timetuple())
  time2 = time1 + 86400
  dfday = df6[(df6["ActivityTimeStamp"] <time2) & (df6["ActivityTimeStamp"] > time1)]
  dfday = dfday.dropna()
  
  listD= []
  for ldap in dfday.LDAP.unique():
    listD.append(getLdap(ldap, dfday))
  
  if (len(listD) > 0):
    finalFrame = pd.DataFrame(listD)[['ldap','startTime','End Time','Total Time']].dropna()
    finalFrameAvg = int(pd.DataFrame(listD)['Total Time'].mean(axis=0))
    averageTimeFormat = convertTime(finalFrameAvg)
  #return ({'LDAP Count': finalFrame[finalFrame['Total Time']> 3600]['ldap'].count(),'Average Time': finalFrameAvg,'Time Format':averageTimeFormat } if (len(listD) > 0) else {'LDAP Count':0,'Average Time':0,'Time Format':0})
  else:
    finalFrame = pd.DataFrame([])

  return finalFrame[finalFrame['Total Time']> 3600] if(len(finalFrame)>0 ) else finalFrame

    

def getLdapDetails(date1):
  time1 = time.mktime(datetime.datetime.strptime(date1+", 6:30:00 AM", "%m/%d/%Y, %I:%M:%S %p").timetuple())
  time2 = time1 + 86400
  dfday = df6[(df6["ActivityTimeStamp"] <time2) & (df6["ActivityTimeStamp"] > time1)]
  dfday = dfday.dropna()
  
  listD= []
  for ldap in dfday.LDAP.unique():
    listD.append(getLdap(ldap, dfday))
  
  if (len(listD) > 0):
    finalFrame = pd.DataFrame(listD)
    return finalFrame[finalFrame['Total Time']> 3600]
  else:
    return pd.DataFrame(listD) 

  

In [25]:
ldapDateSeries = {}
concatdf = []
def ldaDetailsIndateRange(dateRange):
  for date in dateRange:
    print(date)
    ldapDateSeries[date] = getData(date)
    panel= pd.Panel(ldapDateSeries)
  return panel,concatdf

dateRange = pd.date_range('12/1/2018','12/31/2018').strftime('%m/%d/%Y')

dfVal,concatdf = ldaDetailsIndateRange(dateRange)

for date in dateRange:
  concatdf.append(dfVal[date].dropna())
    


12/01/2018


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  if sys.path[0] == '':


12/02/2018
12/03/2018
12/04/2018
12/05/2018
12/06/2018
12/07/2018
12/08/2018
12/09/2018
12/10/2018
12/11/2018
12/12/2018
12/13/2018
12/14/2018
12/15/2018
12/16/2018
12/17/2018
12/18/2018
12/19/2018
12/20/2018
12/21/2018
12/22/2018
12/23/2018
12/24/2018
12/25/2018
12/26/2018
12/27/2018
12/28/2018
12/29/2018
12/30/2018
12/31/2018


In [26]:
def dayperhrcalc(x,y):
  return convertTime(int(int(x)/int(y)))
productivityDataFrame = pd.DataFrame(pd.concat(concatdf).groupby('ldap')['Total Time'].sum())
productivityDataFrame = productivityDataFrame.reset_index()
productivityDataFrame.columns=['ldap', 'Total Time']
productivityDataFrame['Total Days'] = productivityDataFrame['ldap'].apply(lambda x:  (pd.concat(concatdf)['ldap'].value_counts().to_frame()['ldap'][x]) ) 
productivityDataFrame['Time in Hours'] = productivityDataFrame['Total Time'].apply(lambda x: convertTime(int(x)) )
productivityDataFrame['Average Time per day'] = productivityDataFrame.apply(lambda row: dayperhrcalc(row['Total Time'], row['Total Days']), axis=1 )
productivityDataFrame

,ldap,Total Time,Total Days,Time in Hours,Average Time per day
0,achahyun,358838.0,11,99 Hour 40 Min 38 Sec,9 Hour 3 Min 40 Sec
1,achimklai,609843.0,19,169 Hour 24 Min 3 Sec,8 Hour 54 Min 57 Sec
2,ahmadfa,618533.0,20,171 Hour 48 Min 53 Sec,8 Hour 35 Min 25 Sec
3,ajmerasandeep,414689.0,11,115 Hour 11 Min 29 Sec,10 Hour 28 Min 19 Sec
4,alfredjulio,629632.0,20,174 Hour 53 Min 51 Sec,8 Hour 44 Min 41 Sec
5,anhv,637088.0,20,176 Hour 58 Min 8 Sec,8 Hour 50 Min 53 Sec
6,anuarb,499737.0,19,138 Hour 48 Min 56 Sec,7 Hour 18 Min 20 Sec
7,apmrajan,483376.0,17,134 Hour 16 Min 15 Sec,7 Hour 53 Min 53 Sec
8,arahmat,400969.0,12,111 Hour 22 Min 48 Sec,9 Hour 16 Min 53 Sec
9,awyongwee,567638.0,18,157 Hour 40 Min 37 Sec,8 Hour 45 Min 35 Sec


In [36]:
dateRange = pd.date_range('12/1/2018','12/31/2018').strftime('%m/%d/%Y')
monthlyList = []
for date in dateRange:
 # print(date)
  val={}
  val1 = dfVal[date]
  val.update({
    'date':date,
    'LDAP Count': len(dfVal[date].dropna()),
    'Total Time': (0 if (len(dfVal[date].dropna()) ==0) else int((dfVal[date].dropna())['Total Time'].sum())),
    'Total Time Format': convertTime(0 if (len(dfVal[date].dropna()) ==0) else int((dfVal[date].dropna())['Total Time'].sum())),
    'Average Time': (0 if (len(dfVal[date].dropna()) ==0) else int((dfVal[date].dropna())['Total Time'].mean())),
    'Avg Time Format': convertTime(0 if (len(dfVal[date].dropna()) ==0) else int((dfVal[date].dropna())['Total Time'].mean())) 
     })
  monthlyList.append(val)
productivityFrame = pd.DataFrame(monthlyList)[['date','LDAP Count','Total Time','Total Time Format','Average Time','Avg Time Format']]   
 
productivityFrame

,date,LDAP Count,Total Time,Total Time Format,Average Time,Avg Time Format
0,12/01/2018,94,3007793,835 Hour 29 Min 52 Sec,31997,8 Hour 53 Min 16 Sec
1,12/02/2018,74,2366519,657 Hour 21 Min 58 Sec,31979,8 Hour 52 Min 59 Sec
2,12/03/2018,81,2552775,709 Hour 6 Min 14 Sec,31515,8 Hour 45 Min 14 Sec
3,12/04/2018,128,4176196,1160 Hour 3 Min 16 Sec,32626,9 Hour 3 Min 45 Sec
4,12/05/2018,100,3078273,855 Hour 4 Min 32 Sec,30782,8 Hour 33 Min 2 Sec
5,12/06/2018,87,2048191,568 Hour 56 Min 30 Sec,23542,6 Hour 32 Min 21 Sec
6,12/07/2018,112,3676114,1021 Hour 8 Min 33 Sec,32822,9 Hour 7 Min 2 Sec
7,12/08/2018,105,3385026,940 Hour 17 Min 5 Sec,32238,8 Hour 57 Min 18 Sec
8,12/09/2018,73,2291290,636 Hour 28 Min 9 Sec,31387,8 Hour 43 Min 6 Sec
9,12/10/2018,78,2420771,672 Hour 26 Min 11 Sec,31035,8 Hour 37 Min 15 Sec


In [0]:
#Insert into Google Sheet

import gspread 
from gspread_dataframe import get_as_dataframe, set_with_dataframe

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

productivitySheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/18_gAPZPQgiipdApSKlz1XwNxh_cbnvCflKCqZy70Xxk/edit#gid=0')
wks = productivitySheet.worksheet('Sheet1')
for indx in productivityFrame.index.tolist():
  if indx ==0:
    listheader = list(productivityFrame)
    wks.insert_row(listheader, 1) 
  else:
    listrow = list(productivityFrame.iloc[indx-1].as_matrix());
    listrowstr = [str(i) for i in listrow]
    wks.insert_row(listrowstr, indx+1)

wks2 = productivitySheet.worksheet('Sheet2')
for indx in productivityDataFrame.index.tolist():
  if indx ==0:
    listheader = list(productivityDataFrame)
    wks2.insert_row(listheader, 1) 
  else:
    listrow = list(productivityDataFrame.iloc[indx-1].as_matrix());
    listrowstr = [str(i) for i in listrow]
    wks2.insert_row(listrowstr, indx+1)

